In [41]:
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

In [42]:
arg_date = '2021-05-09'
src_format = '%Y-%m-%d'
src_bucket = 'deutsche-boerse-xetra-pds'
trg_bucket = 'xetra-1234'
columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
key = 'xetra_daily_report_' + datetime.today().strftime("%Y%m%d_%H%M%S") + '.parquet'

In [43]:
arg_date_dt = datetime.strptime(arg_date, src_format).date() - timedelta(days=1)

In [44]:
s3 = boto3.resource('s3')
bucket = s3.Bucket(src_bucket)
objects = [obj for obj in bucket.objects.all() if datetime.strptime(obj.key.split('/')[0], src_format).date() >= arg_date_dt]

In [45]:
def csv_to_df(filename):
    csv_obj = bucket.Object(key=filename).get().get('Body').read().decode('utf-8')
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=',')
    return df
df_all = pd.concat([csv_to_df(obj.key) for obj in objects], ignore_index=True)

In [46]:
df_all = df_all.loc[:, columns]

In [47]:
df_all.dropna(inplace=True)

## Get opening price per ISIN and day

In [48]:
df_all['opening_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')

## Get closing price per ISIN and day

In [49]:
df_all['closing_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')

## Aggregations

In [50]:
df_all = df_all.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))

## Percent Change Prev Closing

In [51]:
df_all['prev_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)

In [52]:
df_all['change_prev_closing_%'] = (df_all['closing_price_eur'] - df_all['prev_closing_price']) / df_all['prev_closing_price'] * 100

In [53]:
df_all.drop(columns=['prev_closing_price'], inplace=True)

In [54]:
df_all = df_all.round(decimals=2)

In [55]:
df_all = df_all[df_all.Date >= arg_date]

## Write to S3

In [56]:
out_buffer = BytesIO()
df_all.to_parquet(out_buffer, index=False)
bucket_target = s3.Bucket(trg_bucket)
bucket_target.put_object(Body=out_buffer.getvalue(), Key=key)

s3.Object(bucket_name='xetra-1234', key='xetra_daily_report_20210510_143346.parquet')

## Reading the uploaded file

In [57]:
for obj in bucket_target.objects.all():
    print(obj.key)

xetra_daily_report_20210510_101629.parquet
xetra_daily_report_20210510_143346.parquet


In [60]:
prq_obj = bucket_target.Object(key='xetra_daily_report_20210510_143346.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [61]:
df_report

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,AT00000FACC2,2021-05-10,9.10,8.94,8.94,9.10,78,NaN
1,AT0000606306,2021-05-10,18.75,18.98,18.68,19.01,2104,NaN
2,AT0000609607,2021-05-10,16.48,16.98,16.48,16.98,131,NaN
3,AT0000644505,2021-05-10,112.20,112.80,112.20,113.20,69,NaN
4,AT0000720008,2021-05-10,7.04,7.06,6.99,7.09,2877,NaN
...,...,...,...,...,...,...,...,...
2681,XS2265368097,2021-05-10,15.10,15.14,15.10,15.14,0,NaN
2682,XS2265369574,2021-05-10,22.88,22.86,22.86,22.88,0,NaN
2683,XS2265369731,2021-05-10,10.40,10.50,10.40,10.50,0,NaN
2684,XS2265370234,2021-05-10,24.23,24.58,24.23,24.58,300,NaN
